# Steam market segmentation and predictive analytics

In this notebook, we will load in user information and perform an unsupervised segmentation of the Steam market based on user behavior. Maroe specfically, we will use a non-linear dimensionality reduction technique (tSNE) to visualize how Steam users play games that were released after the year 2010. We will then load in our model that was generated in the "Train model" notebook, and use this model to predict how two different hypothetical games will engage the Steam marketplace. 

In [ ]:
# That's an impressive list of imports.
import numpy as np
from numpy import linalg
from numpy.linalg import norm
from scipy.spatial.distance import squareform, pdist
import pandas as pd
import ast
from collections import Counter

# SQL
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# We import sklearn.
import sklearn
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold.t_sne import (_joint_probabilities,
                                    _kl_divergence)
from sklearn.utils.extmath import _ravel

# We'll use matplotlib for graphics.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib
%matplotlib inline

import re
import ast
from gensim import corpora, models, similarities
from nltk.tokenize import word_tokenize
import pickle

# We import seaborn to make some matrix plots
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})

# We'll generate an animation with matplotlib and moviepy.
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy

# import custom module for processing data
import steamProcessingFunctions as spf
#import steamPlottingFunctions as steamPlot

## Connect to database containing game and user information

In [ ]:
# Define SQL database info
db_name  = 'UserInfo'
username = 'username'
host     = 'localhost'
pwd      = 'password'
port     = '5432'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(username, pwd, host, port, db_name))
print(engine.url)

# connect to database:
con = None
con = psycopg2.connect(database = db_name, user = username, password = pwd, host = host)
cur = con.cursor() # get a cursor to our current connection

## Query users from the database

In [ ]:
numUsers = 15000 # Nuber of users we want to use for tSNE
# make a query to initialize the allPlayers table
create_table_sql = """
SELECT * FROM allPlayers WHERE gameCount IS NOT NULL ORDER BY random() LIMIT {};
""".format(numUsers)
userFrame = pd.read_sql_query(create_table_sql,con)
#userFrame = userFrame.dropna(axis=0, how='any') # Remove any rows that have nans

delCols = ['unnamed', 'avatar', 'avatarfull', 'avatarmedium', 'commentpermission',
       'communityvisibilitystate', 'gameextrainfo', 'gameid', 'gameserverip',
       'gameserversteamid', 'lastlogoff','locstatecode', 'personaname', 'personastate',
       'personastateflags', 'primaryclanid', 'profilestate', 'profileurl',
       'realname','loccityid','lobbysteamid','playtime_total']
userFrame = userFrame.drop(delCols,axis=1)
userFrame = userFrame.dropna(axis=0, how='any') # Remove any rows that have nans
print("processing {} users".format(len(userFrame)))

## Query the information of games that were released after 2010

In [ ]:
# Now let's load in the game data
# make a query to initialize the allGames table
dateLim = 20100000 # let's only consider games that were released after 2010
create_table_sql = """
SELECT * FROM allGames WHERE gamename IS NOT NULL AND releasedate > {};
""".format(dateLim)
gameFrame = pd.read_sql_query(create_table_sql,con)
gameFrame = gameFrame.dropna(axis=0, how='any') # Remove any rows that have nans
games = list(gameFrame['gameid'])

## Bring together user and game information

In the cell below we build two matrices:
- userHasGame is a binary matrix indicating whether each user (row) owns a particular game (column)
- userPlaysGame is similar to userHasGame matrix but contains the amount of time (in hours) each user has played each game

In addition to these matrices, we also build a dataframe containing the user information, that we will process further in subsequent cells to build it into a shape that the model is expecting. 

In [ ]:
# Initialise two matrices with dimensions (users , games)
# We are going to fill one matrix with a binary condition: if the users has each game or not (1 or 0)
# We are going to fill the second matrix with the amount of time (in hours) users play each game
userHasGame    = np.zeros((len(userFrame),len(games)))
userPlaysGame  = np.zeros((len(userFrame),len(games)))

# We are also going to be recording every users first and second most played games, as weel as the amount of time spent playing CS:GO
mostPlayedGame = []
secondMostPlayedGame = []
csgoTime             = []

# Initialise a dataframe with basic user information
testFrame = pd.DataFrame()
testFrame['timecreated'] = userFrame.timecreated
testFrame['usercountry'] = userFrame.loccountrycode
testFrame['games']       = userFrame.games

# Initialise lists to fill dataframe
numFriends   = []
numGames     = []
playTimeList = []
gameIDs      = []

userCount = -1 # counter
for index,row in userFrame.iterrows():
    userCount = userCount+1
    print("Processing user {}".format(userCount),end='\r')
    try: # To account for errors if users don't have any games
        userGames = ast.literal_eval(row['games']) # This is a dict with infor 'appid' and 'playtime_forever' for each game the user ows
        usrAppID  = []
        usrAppT   = []
        for gm in userGames: # Loop through each game the user owns
            usrAppID.append(gm['appid']) # gane ID
            usrAppT.append(gm['playtime_forever']) # Time spent playing the game
        gameCount = -1 # game counter
        for cnt,game in enumerate(games): # Loop through each of the games in our game list
            gameCount = gameCount+1
            if game in usrAppID: # If the user owns that particular game
                ind = usrAppID.index(game) # Index of the game in the user game list
                userHasGame[userCount,gameCount] = 1 # Fill the relevant entry in the userHasGame matrix with 1
                userPlaysGame[userCount,gameCount] = usrAppT[ind] # Fill the relevant entry in the userPlaysGame matrix with the time they have spent playing that game
        # Reorder the game lists based on how much users play
        sortedGames = [x for _,x in sorted(zip(usrAppT,usrAppID))]
        # Note each users most played game, and second most played game
        mostPlayedGame.append(sortedGames[-1])
        if len(sortedGames) > 1:
            secondMostPlayedGame.append(sortedGames[-2])
        else:
            secondMostPlayedGame.append(0)
        #mostPlayedGame.append(usrAppID[usrAppT.index(max(usrAppT))]) # Store the most played game for each user
        
        # Now detect the time spent playing CS:GO
        if 730 in usrAppID:
            ind = usrAppID.index(730)
            csgoTime.append(np.log10(usrAppT[ind]/60))
        else:
            csgoTime.append(0)
        
        # This part grabs the useser info
        playTime   = 0 # Default
        gamesOwned = []
        try:
            friendDict = ast.literal_eval(row['friends'])
            numFriends.append(len(friendDict))
        except:
            numFriends.append(0) # If code throws an error, it is because user has no games
        try:
            gameDict = ast.literal_eval(row['games'])
            numGames.append(len(gameDict))
            for g in gameDict:
                playTime += g['playtime_forever']/60 # divide by 60 to get time expressed in hours
                gamesOwned.append(g['appid'])
            gameIDs.append(gamesOwned)
        except:
            numGames.append(0) # If code throws an error, it is because the user has no games
            gameIDs.append(0)
        playTimeList.append(playTime)    
        
        
    except:
        pass
    
# Now we fill in the columns of the dataframe with the lists we just compted
testFrame['friends']    = numFriends
testFrame['playTime']   = playTimeList
testFrame['numGames']   = numGames
testFrame['gamesOwned'] = gameIDs  

## Load in the tf-idf models for game tag and descriptions

These models were generated in a separate script called "Train TF-IDF model (tags)" and "Train TF-IDF model (description)". These models will be used to engineer the similarity of the hypothetical games to the games that each user currently owns. 

In [ ]:
# Load in the pretrained tf-idf models,dictionary, and corpus for game tags and game descriptions
tag_dictionary = corpora.Dictionary.load('/home/gameTagDict.dict')
tag_corpus     = corpora.MmCorpus('/home/gameTagCorpus.mm')
description_dictionary = corpora.Dictionary.load('/home/gameDescriptionDict.dict')
description_corpus     = corpora.MmCorpus('/home/gameDescriptionCorpus.mm')

# Initialise a term-frequency inverse document frequency model based on the corpus for both tags and descriptions
tag_tfidf = models.TfidfModel(tag_corpus) 
description_tfidf =  models.TfidfModel(description_corpus)
# Initialise a similarity index using the stored index, corpus, and dictionary
tag_sims = similarities.Similarity('/home/',
                                   tag_tfidf[tag_corpus],num_features=len(tag_dictionary))
description_sims = similarities.Similarity('/home/',
                    description_tfidf[description_corpus],num_features=len(description_dictionary))

## Load in the Half-Life 3 and Euro Trucker 3 game hypothetical game features

In [ ]:
hl3frame = pd.read_csv('/home/hl3frame')
del hl3frame['Unnamed: 0']
euroTruck3 = pd.read_csv('/home/euroTruck3')
del euroTruck3['Unnamed: 0']

## Define functions to compute tag and description similarity

In [ ]:
allGames = gameFrame.gameid.tolist()
def compute_tag_similarity(testRow,userGameDict):
    # This function comutes the similarity of one game to the games a user already owns based on game tags
    tmp         = testRow.tags.tolist()
    currentTags = ast.literal_eval(tmp[0]) # Evaluate to get in list form
    query       = ' '.join(currentTags) # Join all together with spaces

    # Tokenize
    query_doc = [w.lower() for w in word_tokenize(query)]
    # Compare to dictionary we computed on all other games
    query_doc_bow = tag_dictionary.doc2bow(query_doc)
    # Vectorize word representation
    query_doc_tf_idf = tag_tfidf[query_doc_bow]
    # Compute the similarity of the game with current tags to al games in our database
    query_similarity = tag_sims[query_doc_tf_idf]

    # Control for if the user has only one game
    if type(userGameDict) == dict:
        userGameDict = list([userGameDict])
        
    gamesOwned  = []
    gamesPlayed = []
    for g in userGameDict:
        if g['appid'] in allGames:
            #if g['appid'] != testGame: # If the user owns the testGame, we should omit this from the model
            gamesOwned.append(g['appid'])
            gamesPlayed.append(g['playtime_forever']+0.01) # Add small amount to avoid dividing by zero
    # normalize the playtime to the total amount of playtime
    gamesPlayed = np.array([x / sum(gamesPlayed) for x in gamesPlayed]) 
    
    
    # We only want to compare to games that are in our original list
    testInd = []
    for game in gamesOwned:
        if game in allGames:
            testInd.append(allGames.index(game))
    user2game_similarity = query_similarity[testInd]
    user2game_weighted_similarity = user2game_similarity*gamesPlayed
    
    # If we just have empty arrays, then fill with 0
    if user2game_similarity.size < 1:
        user2game_similarity = 0
        user2game_weighted_similarity = 0
        
    return np.mean(user2game_similarity)#, np.median(user2game_weighted_similarity)

def preprocess_descriptions(description):
    # Define a list of words that we will not include, since they do not convey meaning
    try:
        removelist = set('for a of the and to in on is or be as where it its at an - with by'.split())
        description = re.sub(r"\s+", " ", description) # remove line breaks and tabs
        description = re.sub(r"<.*?>", "", description) # remove <>
        description = description.replace(",","") # remove commas
        description = description.replace(".","") # remove periods
        # Split text into a list of words
        text = [word for word in description.lower().split() if word not in removelist]
        return text
    except:
        return []

def compute_description_similarity(testRow,userGameDict):
    # This function comutes the similarity of one game to the games a user already owns based on game description
    tmp         = testRow.detailedescription.tolist()
    query_doc   = preprocess_descriptions(tmp[0]) # preprocess data

    # Compare to dictionary we computed on all other games
    query_doc_bow = description_dictionary.doc2bow(query_doc)
    # Vectorize word representation
    query_doc_tf_idf = description_tfidf[query_doc_bow]
    # Compute the similarity of the game with current tags to al games in our database
    query_similarity = description_sims[query_doc_tf_idf]
    
    # Control for if the user has only one game
    if type(userGameDict) == dict:
        userGameDict = list([userGameDict])
    
    gamesOwned  = []
    gamesPlayed = []
    for g in userGameDict:
        if g['appid'] in allGames:
            #if g['appid'] != testGame: # If the user owns the testGame, we should omit this from the model
            gamesOwned.append(g['appid'])
            gamesPlayed.append(g['playtime_forever']+0.01)
    # normalize the playtime to the total amount of playtime
    gamesPlayed = np.array([x / sum(gamesPlayed) for x in gamesPlayed]) 
    
    # We only want to compare to games that are in our original list
    testInd = []
    for game in gamesOwned:
        if game in allGames:
            testInd.append(allGames.index(game))
    user2game_similarity = query_similarity[testInd]
    user2game_weighted_similarity = user2game_similarity*gamesPlayed
    
    # If we just have empty arrays, then fill with 0
    if user2game_similarity.size < 1:
        user2game_similarity = 0
        user2game_weighted_similarity = 0
    
    return np.median(user2game_similarity)#, np.median(user2game_weighted_similarity)

## Compute the similarity of hypothetical games to users games

This code loops through each user and computes the tf-idf similarity of half-life 3 and euro trucker 3 tags and description to the games each user currently owns. The code calls a separate function that can be found above 

In [ ]:
# Now let's compute the similarity of hypothetical games to users currently owned games
hl3_tag = []
hl3_des = []
et3_tag = []
et3_des = []
allGames = gameFrame.gameid.tolist()
for index,user in testFrame.iterrows():
    gd = ast.literal_eval(user.games)
    hl3_tag.append(compute_tag_similarity(hl3frame,gd))
    hl3_des.append(compute_description_similarity(hl3frame,gd))
    et3_tag.append(compute_tag_similarity(euroTruck3,gd))
    et3_des.append(compute_description_similarity(euroTruck3,gd))
      
    print("Processing user {}".format(index),end='\r')

## Bring together user and hypothetical game information in the same dataframe

In the cell below, I am making new dataframes with only the variables that I am interested in for both half-life 3 and eurotrucker 3, and inserting the similarity scores that I computed in the cell above. 

In [ ]:
# Let's make the half-life 3 dataframe first
hframe    = pd.concat([hl3frame[['developers','final_price','init_price','metacriticscore','releasedate','recommendations','genre']]]*len(testFrame),axis=0)
hframe    = hframe.reset_index()
tf        = testFrame
tf        = tf.reset_index()
df_hl3    = pd.concat([tf,hframe],axis=1)
df_hl3['description_similarity'] = hl3_des
df_hl3['tag_similarity'] = hl3_tag  

# Now let's make the dataframe for the eurotruck simulator 3
eframe    = pd.concat([euroTruck3[['developers','final_price','init_price','metacriticscore','releasedate','recommendations','genre']]]*len(testFrame),axis=0)
eframe    = eframe.reset_index()
tf        = testFrame
tf        = tf.reset_index()
df_et3    = pd.concat([tf,eframe],axis=1)
df_et3['description_similarity'] = et3_des
df_et3['tag_similarity'] = et3_tag  

## Process dataframes 

The code below processes dataframes such that they are almost ready for the model. We need to turn categorical variables into dummy variables (new columns consisting of 0's and 1's) that the model can handle. In addition we have to perform some other preprocessing steps such that the XGBoost model can handle the data, this includes removing any strange characters (,[]<>) from columns names, and changing certain columns datatype to float. 

In [ ]:
def preprocess_columns(df):
    # We also have to remove any weird characters from the dataframe column names
    df.columns = [col.replace(',', '') for col in df.columns]
    df.columns = [col.replace('[', '') for col in df.columns]
    df.columns = [col.replace(']', '') for col in df.columns]
    df.columns = [col.replace('<', '') for col in df.columns]
    return df

def process_frame(df):
    
    # Generate dummy dataframes from our categorical variables    
    dfCountry   = pd.get_dummies(df['usercountry'])
    dfCountry   = preprocess_columns(dfCountry)
    dfDeveloper = pd.get_dummies(df['developers'])
    dfDeveloper = preprocess_columns(dfDeveloper)
    dfGenre     = pd.get_dummies(df['genre'])
    dfGenre     = preprocess_columns(dfGenre)

    # Concatinate the dummy variables with the original dataframe
    df_new      = pd.concat([df, dfCountry, dfDeveloper, dfGenre], axis=1)
    # Remove the original categorical columns from the dataframe, keeping only the dummy variable columns
    df_new = df_new.drop(['usercountry','developers','genre','tags'],axis=1)

    # We have to convert the data type to float so that the XGBoost model can handle it
    df_new['timecreated'] = pd.to_numeric(df_new['timecreated'],downcast='float')
    df_new['friends'] = pd.to_numeric(df_new['friends'],downcast='float')
    df_new['playTime'] = pd.to_numeric(df_new['playTime'],downcast='float')
    df_new['numGames'] = pd.to_numeric(df_new['numGames'],downcast='float')
    return df_new

# Process the dataframes by creating dummy variabels etc.
df_et3 = process_frame(df_et3)
df_hl3 = process_frame(df_hl3)

## Get dataframes ready for passing to the model

The model is expecting our data to be in a certain shape, with a particular number of columns with certain names, etc. The code below now takes our current dataframes for half-life 3 and euro trucker 3 and removed excess columns that were not in the training set, and adds those that are missing in the current dataframes. 

In [ ]:
def add_missing_dummy_columns( d, columns ):
    missing_cols = set( columns ) - set( d.columns )
    for c in missing_cols:
        d[c] = 0

def fix_columns( d, columns ):  

    add_missing_dummy_columns( d, columns )

    # make sure we have all the columns we need
    assert( set( columns ) - set( d.columns ) == set())

    extra_cols = set( d.columns ) - set( columns )
    if extra_cols:
        print("extra columns: {}".format(extra_cols))

    d = d[ columns ]
    return d

# Now we are going to load in some rows from the dataframe that was used to train the model.
testSet  = pd.read_csv('/home/trainingDataset',nrows=20)
del testSet['userHasGame']
del testSet['Unnamed: 0']
testCols = testSet.columns

# Using the above functions, we will ensure that the test dataframes are in the shape the model is expecting
df_et3 = fix_columns(df_et3,testCols)
df_hl3 = fix_columns(df_hl3,testCols)

## Load in the model and predict how each user will respond to both games

In [ ]:
# Load in the model 
modelList = pickle.load(open('/home/ensemble_model.sav', 'rb'))

In [ ]:
hl3_pred  = np.zeros((len(modelList),len(df_hl3)))
et3_pred  = np.zeros((len(modelList),len(df_et3)))
hl3_prob  = np.zeros((len(modelList),len(df_hl3)))
et3_prob  = np.zeros((len(modelList),len(df_et3)))
for ind,model in enumerate(modelList):
    tmp = model.predict_proba(df_hl3)
    hl3_prob[ind,:] = tmp[:,1]
    tmp = model.predict_proba(df_et3)
    et3_prob[ind,:] = tmp[:,1]
    et3_pred[ind,:] = model.predict(df_et3)
    hl3_pred[ind,:] = model.predict(df_hl3)

In [ ]:
hl3_predictions = np.round(np.median(hl3_pred,axis=0))
et3_predictions = np.round(np.median(et3_pred,axis=0))
hl3_probabilities = np.mean(hl3_prob,axis=0)
et3_probabilities = np.mean(et3_prob,axis=0)

## Compute the fraction of users that are predicted to purchase each game

In [ ]:
et3_percentage = round((sum(et3_predictions)/len(et3_predictions))*100,3)
hl3_percentage = round((sum(hl3_predictions)/len(hl3_predictions))*100,3)
print("{}% of users are predicted to buy Euro Trucker 3 Simulator".format(et3_percentage))
print("{}% of users are predicted to buy Half-Life 3".format(hl3_percentage))

## Plot the probability distributions of how likely each user is to buy both games


In [ ]:
f  = plt.figure(figsize=(11, 5))
ax = plt.subplot()
    
# Make histogram of half life 3 sales probability distribution:
weights = np.ones_like(hl3_probabilities)/float(len(hl3_probabilities))
n, bins, patches = plt.hist(hl3_probabilities, bins=np.linspace(0,1,100), 
                            weights=weights, facecolor='blue', alpha=0.5,label='Half-Life 3')
# Make histogram of Eurotruck 3 sales probability distribution:
weights = np.ones_like(et3_probabilities)/float(len(et3_probabilities))
n, bins, patches = plt.hist(et3_probabilities, bins=np.linspace(0,1,100),
                            weights=weights,facecolor='red', alpha=0.5,label='Euro Truck Simulator 3')

ax.legend(bbox_to_anchor=(0.6, 1), loc=2, borderaxespad=0.)
ax.set_xlim(0,1)
ax.set_xlabel('Probability users will buy game',fontsize=20)
ax.set_ylabel('Fraction of users',fontsize=20)
plt.show()

## Define a funciton that will plot t-SNE results

In [ ]:
# Define a functiont to perform a scatter plot on most played game
def tSNE_scatter(x,ng,gameNames,gameID,mostPlayedGame,dotSize):
    # Prepare the plot
    f  = plt.figure(figsize=(8, 8),dpi=100)
    ax = plt.subplot(aspect='equal')
    palette = np.array(sns.color_palette("hls", ng)) # produce some nice colors for the scatterplot
    
    groupedSC = []
    grayColor = np.array([0.75,0.75,0.75]) # We will plot the 'other' datapoints in gray
    # Plot all data points first
    gameInd = [i for i, x in enumerate(mostPlayedGame) if x != gameNames]
    sc = ax.scatter(x[gameInd,0], x[gameInd,1], lw=0, s=20, c=grayColor,label="Other")
    groupedSC.append(sc)
    
    # Now let's loop through and plot each of the most played games in the tSNE space
    colInd = -1
    for game in gameID:
        colInd = colInd + 1
        #print(colInd)
        gameInd = [i for i, x in enumerate(mostPlayedGame) if x == game]
        sc = ax.scatter(x[gameInd,0], x[gameInd,1], lw=0, s=dotSize, c=palette[colInd],label=gameNames[colInd])
        groupedSC.append(sc)
    ax.legend(bbox_to_anchor=(1.05, 1), loc=0, borderaxespad=0.,fontsize=20)
    #ax.legend()
    ax.axis('off')
    ax.axis('tight') 
    return f, groupedSC, ax

## Plot how much time users spend playing games

Below I am just plotting the userPlaysGame matrix, which contains the amount of time (in hours) each user has played a list of games that has been released since 2010. You can see that the matrix is generally quite sparse, but one can observe both vertical and horizontal 'stripes' in the matrix indicating that users tend to play similar games a lot (vertical stripe), and certain usrs like to play a lot of games (horizontal stripes).

In [ ]:
num = 500
testMat = userPlaysGame[:num,:]

f  = plt.figure(figsize=(10, 5),dpi=100)
sc = sns.heatmap(np.log10(testMat),vmin=0,vmax=3,cmap="Blues",
                 cbar_kws={'label': 'log$_{10}$(hours played)'}, yticklabels=False,xticklabels=False)
sc.set_xlabel('Games',size=25)
sc.set_ylabel('User',size=25)
f.show()

## Compute t-SNE on user playing time and game ownership matrices

Here is a description of t-SNE that I lifted from the sklearn webpage:

"t-SNE [1] is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results."


In [ ]:
RS = 20170912 # random initialization so we ge the same results each time we run the analysis
positions = [] # This variable will be filled upon calling the TSNE function
tSNE_playtime = TSNE(random_state=RS,perplexity=30).fit_transform(userPlaysGame)
print("Finished userPlaysGame tSNE")

## Compute a list for most played games on Steam

We are going to compute a list of most played games across all of the users in our dataframe. This will be helpful for visualization of the Steam marketplace later when we plot the t-SNE

In [ ]:
count = Counter(mostPlayedGame)
mpg   = count.most_common() # tuple of most played games
cnt      = 0 # counter
numGames = 20 # Number of games to plot in tSNE space
gameNames = []
gameID    = [] # Most played game
for game in mpg:
    if not gameFrame.loc[gameFrame['gameid'] == game[0]].empty: # Some app's are not games, and have empty fields in the pandas dataframe. So we skip those
        cnt = cnt + 1
        if cnt > numGames: # We only want a certain number of games
            break
        
        # Store the game name and appid for each game in the top 20 most played list
        tmp = gameFrame.loc[gameFrame['gameid'] == game[0]]['gamename']
        gameNames.append(tmp.iloc[0])
        gameID.append(game[0])

In [ ]:
gl = np.array([1,3,4,5,7])
top5_games = []
top5_names = []
for gi in gl:
    top5_games.append(gameID[gi])
    top5_names.append(gameNames[gi])
testMPG = []
for usr in mostPlayedGame:
    if usr in top5_games:
        testMPG.append(usr)
    else:
        testMPG.append(0)

f,sc,ax = tSNE_scatter(tSNE_playtime,5,top5_names,top5_games,testMPG,80)

## Plot the t-SNE space and label each user by their most played game

In [ ]:
f,sc,ax = tSNE_scatter(tSNE_playtime,numGames,gameNames,gameID,mostPlayedGame,30)

## Plot users second most played game as well

Looking at the above plot it is apparent that most users favourite game is Counter Strike, however we still observe several clusters of (predominantly) Counter Strike users. To further break down this market representation of Steam user behavior, I am plotting below the second most played game for users who play Counter Strike the most. As you can see, this reveals more finer details about Steam user behavior. For example, the cluster in the lower right quadrant seems to be a group of users that play counter strike the most, but also play GTA V the second most. 

In [ ]:
# We can make another list of games, where for users who play CS:GO the most we plot their second favourite game
s = []
for i,u in enumerate(mostPlayedGame):
    if u == 730:
        s.append(secondMostPlayedGame[i])
    else:
        s.append(u)
f,sc,ax = tSNE_scatter(tSNE_playtime,numGames,gameNames,gameID,s,30)

## Plot counter strike play time in t-SNE space

The above plot is informative, but it does not go all the way to explaining why we have such diversity in the t-SNE space when it comes to Counter Strike users. To get a better idea about these users behavior, I am plotting the t-SNE space, where the color of each dot indicates the total time each user has spent playing Counter Strike. Interetingly, it seems that the many of the cluster of CS:GO playes can be partially explained by the amount of time each player has spent playing that game (you can see by the consistent colours within each cluster)

In [ ]:
f  = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.subplot(aspect='equal') 
sc = ax.scatter(tSNE_playtime[:,0], tSNE_playtime[:,1], lw=0, s=15, c=csgoTime,cmap='plasma',vmin=-1,vmax=4)

cbar = plt.colorbar(sc,orientation='horizontal',shrink=0.5)
#cbar.ax.set_xticklabels(['Low', 'Medium', 'High'])  # horizontal colorbar
cbar.set_label('log$_{10}$(hours playing CS:GO)')
ax.axis('off')
ax.axis('tight') 
plt.show() 

## Project sales predictions into t-SNE space

To unserstand how Half-Life 3 will engage the Steam marketplace, we can plot the t-SNE space again, but this time colour coding each datapoint with the probability that each user will buy the hypothetical game. As you can see below, Half-Life 3 is predicted to engage a large proportion of the Steam marketplace, with the exception of a small cluster of users to the left of the plot (these are users that don't buy or play many games). 

In [ ]:
f  = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.subplot(aspect='equal') 
sc = ax.scatter(tSNE_playtime[:,0], tSNE_playtime[:,1], lw=0, s=20, c=hl3_probabilities,cmap='viridis',vmin=0.40,vmax=0.60)

cbar = plt.colorbar(sc,orientation='horizontal',shrink=0.5,ticks=[0.4,0.5,0.6])
#cbar.ax.set_xticklabels(['Low', 'Medium', 'High'])  # horizontal colorbar
cbar.set_label('Probability user will buy Half-life 3')
ax.axis('off')
ax.axis('tight') 
plt.show() 

## Project Euro Trucker Simulator 3 sales into t-SNE space

As can be seen in the plot below, when we do the same for Euro Trucker 3 we see only very sparse engagement in the Steam market. 

In [ ]:
f  = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.subplot(aspect='equal') 
sc = ax.scatter(tSNE_playtime[:,0], tSNE_playtime[:,1], lw=0, s=20, c=et3_probabilities,cmap='viridis',vmin=0.3,vmax=0.5)

cbar = plt.colorbar(sc,orientation='horizontal',shrink=0.5,ticks=[0.3,0.4,0.5])
#cbar.ax.set_xticklabels(['Low', 'Medium', 'High'])  # horizontal colorbar
cbar.set_label('Probability user will buy Euro Trucker Simulator 3')
ax.axis('off')
ax.axis('tight') 
plt.show() 

## Business case: explore parameters to maximize profit

One of the tough decisions for game developers is to decide how to price each game. There needs to be a balance between the game price, the number of users who will be engaged, and of course profit. To illustrate how the tools I have developed can be used by game developers, I am going to vary the price of a theoretical Euro Trucker 3 game, while keeping everything else constant, and predict how many users will buy the game. 

In [ ]:
n = 6
prices = np.linspace(5,30,n)
df_et3.head()

score = np.zeros(n)

testF = df_et3
count = -1
for price in prices:
    count = count + 1
    testF['final_price'] = price
    testF['init_price'] = price
    
    tmp_pred  = np.zeros((len(modelList),len(df_hl3)))
    for ind,model in enumerate(modelList):
        tmp_pred[ind,:] = model.predict(testF)    
    tmp_predictions = np.round(np.median(tmp_pred,axis=0))
    score[count] = sum(tmp_predictions)

## Plot game sales vs revenue

It turns out that although more games will be sold if the game is priced at $10, the actual revenue for the developer studio is maximized when the game is sold at $15. 

In [ ]:
numSteamUsers = 125000000
percent_users = score/len(testF)
revenue = (percent_users*numSteamUsers)*prices

fig, ax1 = plt.subplots(figsize=(8,5),dpi=100)
t = np.arange(0.01, 10.0, 0.01)
ax1.plot(prices, percent_users, 'b-',label='Predicted game sales')
ax1.set_xlabel('Euro Trucker Simulator 3 price ($)',fontsize=20)
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('% of all Steam users', color='b',fontsize=20)
ax1.tick_params('y', colors='b')
ax1.grid()
ax1.legend(loc=3)

ax2 = ax1.twinx()
ax2.plot(prices, revenue/1000000, 'r',label='Projected revenue')
ax2.set_ylabel('Dollars in millions', color='r',fontsize=20)
ax2.tick_params('y', colors='r')
ax2.grid()
ax2.legend(loc=1)

fig.tight_layout()
plt.show()

## Monkey patch the gradient descent script to store each iteration

Here we are going to monkey patch the _gradient_descent function from the tSNE package in python. 
We will only change it very slightly, by adding one line to store the location of each data point
across iterations - so we can track how the algorithm optimizes the separation of data points


In [ ]:

def _gradient_descent(objective, p0, it, n_iter, n_iter_without_progress=50,
                      momentum=0.5, learning_rate=200.0, min_gain=0.01,
                      min_grad_norm=1e-7, min_error_diff=1e-7, verbose=0,
                      n_iter_check=100,kwargs=None,objective_error=None, args=[]):
    if args is None:
        args = []
    if kwargs is None:
        kwargs = {}

    p = p0.copy().ravel()
    update = np.zeros_like(p)
    gains = np.ones_like(p)
    error = np.finfo(np.float).max
    best_error = np.finfo(np.float).max
    best_iter = 0

    for i in range(it, n_iter):
        new_error, grad = objective(p, *args, **kwargs)
        grad_norm = linalg.norm(grad)

        inc = update * grad >= 0.0
        dec = np.invert(inc)
        gains[inc] += 0.05
        gains[dec] *= 0.95
        np.clip(gains, min_gain, np.inf)
        grad *= gains
        update = momentum * update - learning_rate * grad
        p += update
        
        # This is the line we are adding to the 
        # function to store data point positions
        #######################################
        #######################################
        positions.append(p.copy())  
        #######################################
        #######################################
        
        if (i + 1) % n_iter_check == 0:
          
            if new_error is None:
                new_error = objective_error(p, *args)
            error_diff = np.abs(new_error - error)
            error = new_error

            if verbose >= 2:
                m = "[t-SNE] Iteration %d: error = %.7f, gradient norm = %.7f"
                print(m % (i + 1, error, grad_norm))

            if error < best_error:
                best_error = error
                best_iter = i
            elif i - best_iter > n_iter_without_progress:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: did not make any progress "
                          "during the last %d episodes. Finished."
                          % (i + 1, n_iter_without_progress))
                break
            if grad_norm <= min_grad_norm:
                if verbose >= 2:
                    print("[t-SNE] Iteration %d: gradient norm %f. Finished."
                          % (i + 1, grad_norm))
                break
            if error_diff <= min_error_diff:
                if verbose >= 2:
                    m = "[t-SNE] Iteration %d: error difference %f. Finished."
                    print(m % (i + 1, error_diff))
                break

        if new_error is not None:
            error = new_error

    return p, error, i
sklearn.manifold.t_sne._gradient_descent = _gradient_descent

# Recompute t-SNE

In [ ]:
RS = 20170912 # random initialization so we ge the same results each time we run the analysis
positions = [] # This variable will be filled upon calling the TSNE function
tSNE_playtime = TSNE(random_state=RS,perplexity=30).fit_transform(userPlaysGame)
print("Finished userPlaysGame tSNE")
#tSNE_owns = TSNE(random_state=RS).fit_transform(userHasGame)
#print("Finished userHasGame tSNE")

## Generate animation of t-SNE computation and save to file

In [ ]:
X_iter = np.dstack(position.reshape(-1, 2) for position in positions)

# Plot the first frame with the
#ff, sc, ax = tSNE_scatter(X_iter[...,-1],numGames,gameNames,mostPlayedGame)
ff, sc, ax = tSNE_scatter(X_iter[...,-1],5,top5_names,top5_games,testMPG,80)

def make_frame_tsne(t):
    
    # Get the indices for data points for the 'other' condition
    gameInd = [i for i, x in enumerate(mostPlayedGame) if x != gameNames]
    i = int(t*40) # skip factor
    x = X_iter[gameInd,:,i] # Pull out data
    sc[0].set_offsets(x)

    # Now let's loop through and plot each of the most played games in the tSNE space
    colInd = 0
    for game in top5_games:
        colInd = colInd + 1
        gameInd = [i for i, x in enumerate(mostPlayedGame) if x == game]
        x = X_iter[gameInd,:,i]
        sc[colInd].set_offsets(x)

    return mplfig_to_npimage(ff)

animation = mpy.VideoClip(make_frame_tsne,duration=X_iter.shape[2]/40.)
animation.write_gif("/home/tSNE_animation_final.gif", fps=20)